In [1]:
#!pip install Augmentor 

In [1]:
#ライブラリをインポート
#標準ライブラリ
import os
import pathlib

#サードパーティー製
import Augmentor
import pandas as pd

# 概要

Augmenterを使用して、オフライン拡張をおこなう．<br>
実験方法は様々あるが、付けたし実験を行い、その水増し処理が適切な検証をおこなう．

計5種類の水増し処理を実施し、一先ず・・・水増し枚数は1処理×300+300枚で固定する．
1. 画像の回転
2. 明るさの変換
3. 彩度の変換
4. 歪ませる
5. 切り抜き
6. 反転
7. 弾性歪み

In [2]:
"""
変更箇所
"""
#訓練画像のパス
train_path = "Train"
#水増し処理後の画像の保存先のパス
augmentation_output_path = "Train/output"
#名前変更後の画像の出力先のパス
rename_output_path = "output_after_rename_img_"

In [3]:
#出力先の画像の名前を求める形に変更
def rename_path(augmentation_output_path, rename_output_path):
    """
    水増し処理をした画像のパス名を変換させる関数
    
    augmentation_output_path : str
        水増し処理後の画像群のパス
    rename_output_path : str
        リネーム後の画像を出力するパス
    """
    
    #画像のパスを取得
    list_path = list(pathlib.Path(augmentation_output_path).iterdir())
    #画像のパス名をstr型に変更
    list_path = [str(path) for path in list_path]
    
    #DF化
    df_list_path = pd.DataFrame(list_path)
    
    #それぞれのパスのフルーツ名を取得
    list_fruit_name = list(df_list_path[0].str.split("\\", expand = True).iloc[:, -1].str.split("_", expand = True).iloc[:, 2])
    
    #パスと果物名の列があるdfを作成
    df_path_and_fruitname = pd.DataFrame([list_path, list_fruit_name]).T
    df_path_and_fruitname.columns = ["パス", "果物名"]
    
    #パス名の変換を実施
    apple_count = 0
    tomato_count = 0
    cherry_count = 0
    for path, fruit_name in df_path_and_fruitname.values:
        #この書き方あまり好きではないが、、、時短のため
        #リンゴの場合
        if(fruit_name == "apple"):
            #名前の変更
            os.rename(path, os.path.join(rename_output_path, fruit_name + '_' + str(apple_count) + ".jpg"))
            #リンゴ＋１
            apple_count += 1
        #トマトの場合
        elif(fruit_name == "tomato"):
            #名前の変更
            os.rename(path, os.path.join(rename_output_path, fruit_name + '_' + str(tomato_count) + ".jpg"))
            #トマト＋１
            tomato_count += 1
        #サクランボの場合
        elif(fruit_name == "cherry"):
            #名前の変更
            os.rename(path, os.path.join(rename_output_path, fruit_name + '_' + str(cherry_count) + ".jpg"))
            #さくらんぼ＋１
            cherry_count += 1

## 画像の回転

In [4]:
def rotate(img_path, prob = 0.9, max_left = 180, max_right = 180):
    """
    画像を回転させる水増し処理
    
    img_path:str
        水増し処理を行いたい画像群のパス
    prob:double
        画像を回転させる確率
    max_left:int
        左に傾ける角度の最大値
    max_right:int
        右に傾ける角度の最小値
    """
    
    # パイプラインを作成する。
    p = Augmentor.Pipeline(img_path)
    
    # ランダムな回転を実施
    p.rotate_without_crop(probability = prob, max_left_rotation = max_left, max_right_rotation = max_right)
    
    p.sample(3000)

In [5]:
rotate(train_path)

Initialised with 999 image(s) found.
Output directory set to Train\output.

Processing <PIL.Image.Image image mode=RGB size=510x340 at 0x14A04F58E20>: 100%|█| 3000/3000 [00:10<00:00, 281.39 Sampl


In [6]:
#適切な形にパス名を変更
rename_path(augmentation_output_path, rename_output_path)

## 画像の回転＋明るさ

In [9]:
# 回転と明るさの水増し処理
def rotate_and_adjust(img_path, prob = 0.5, max_left = 25, max_right = 25, min_factor_adjust = 0.7, max_factor_adjust = 1.7):
    """
    画像を回転させる水増し処理
    
    img_path:str
        水増し処理を行いたい画像群のパス
    prob:double
        画像を回転させる確率
    max_left:int
        左に傾ける角度の最大値
    max_right:int
        右に傾ける角度の最小値
    min_factor_adjust:double
        明るさの最小値
    max_factor_adjust:double
        明るさの最大値
    
    """
    
    # パイプラインを作成する。
    p = Augmentor.Pipeline(img_path)
    
    # ランダムな回転を実施
    p.rotate_without_crop(probability = prob, max_left_rotation = max_left, max_right_rotation = max_right)
    #明るさの変更
    p.random_brightness(probability=prob, min_factor=min_factor_adjust, max_factor=max_factor_adjust)
    
    p.sample(3000)

In [10]:
rotate_and_adjust(train_path)
#適切な形にパス名を変更
rename_path(augmentation_output_path, rename_output_path)

Initialised with 999 image(s) found.
Output directory set to Train\output.

Processing <PIL.Image.Image image mode=RGB size=120x120 at 0x14A05688C70>: 100%|█| 3000/3000 [00:14<00:00, 204.98 Sampl


## 画像の回転＋明るさ＋彩度

In [7]:
# 回転と明るさと彩度の水増し処理
def rotate_and_adjust_saturation(img_path, prob = 0.5, max_left = 25, max_right = 25, min_factor_adjust = 0.3, max_factor_adjust = 1.7, \
                                 min_factor_saturation = 0.3, max_factor_saturation = 1.7):
    """
    画像を回転させる水増し処理
    
    img_path:str
        水増し処理を行いたい画像群のパス
    prob:double
        画像を回転させる確率
    max_left:int
        左に傾ける角度の最大値
    max_right:int
        右に傾ける角度の最小値
    min_factor_adjust:double
        明るさの最小値
    max_factor_adjust:double
        明るさの最大値
    min_factor_saturation:double
        彩度の最小値
    max_factor_saturation:double
        彩度の最大値
    
    """
    
    # パイプラインを作成する。
    p = Augmentor.Pipeline(img_path)
    
    # ランダムな回転を実施
    p.rotate(probability = prob, max_left_rotation = max_left, max_right_rotation = max_right)
    #明るさの変更
    p.random_brightness(probability=prob, min_factor=min_factor_adjust, max_factor=max_factor_adjust)
    #彩度の変更
    p.random_color(probability=prob, min_factor=min_factor_saturation, max_factor=max_factor_saturation)
    
    p.sample(3000)

In [7]:
rotate_and_adjust_saturation(train_path)
#適切な形にパス名を変更
rename_path(augmentation_output_path, rename_output_path)

Processing <PIL.Image.Image image mode=RGB size=162x180 at 0x17DC726AFD0>:   1%| | 7/1200 [00:00<00:33, 36.05 Samples/s

Initialised with 603 image(s) found.
Output directory set to ../../Desktop/Train\output.

Processing <PIL.Image.Image image mode=RGB size=276x183 at 0x17DC87C9F60>: 100%|█| 1200/1200 [00:03<00:00, 330.41 Sampl


## 画像の回転＋明るさ＋彩度＋歪み

In [39]:
# 画像の回転＋明るさ＋彩度＋歪み
def rotate_and_adjust_saturation_skrw(img_path, prob = 0.5, max_left = 25, max_right = 25, min_factor_adjust = 0.3, max_factor_adjust = 1.7, \
                                 min_factor_saturation = 0.3, max_factor_saturation = 1.7, mag = 1):
    """
    画像を回転させる水増し処理
    
    img_path:str
        水増し処理を行いたい画像群のパス
    prob:double
        画像を回転させる確率
    max_left:int
        左に傾ける角度の最大値
    max_right:int
        右に傾ける角度の最小値
    min_factor_adjust:double
        明るさの最小値
    max_factor_adjust:double
        明るさの最大値
    min_factor_saturation:double
        彩度の最小値
    max_factor_saturation:double
        彩度の最大値
    mag:double
        歪みの程度（おそらく）
    
    """
    
    # パイプラインを作成する。
    p = Augmentor.Pipeline(img_path)
    
    # ランダムな回転を実施
    p.rotate(probability = prob, max_left_rotation = max_left, max_right_rotation = max_right)
    #明るさの変更
    p.random_brightness(probability=prob, min_factor=min_factor_adjust, max_factor=max_factor_adjust)
    #彩度の変更
    p.random_color(probability=prob, min_factor=min_factor_saturation, max_factor=max_factor_saturation)
    #歪みを実施
    p.skew(probability=prob, magnitude=mag)
    
    p.sample(20000)

In [40]:
rotate_and_adjust_saturation_skrw(train_path)
#適切な形にパス名を変更
rename_path(augmentation_output_path, rename_output_path)

Executing Pipeline:   0%|                                                              | 0/20000 [00:00<?, ? Samples/s]

Initialised with 603 image(s) found.
Output directory set to ../../Desktop/Train\output.

Processing <PIL.Image.Image image mode=RGB size=269x196 at 0x17DD63EF3C8>: 100%|█| 20000/20000 [03:33<00:00, 93.89 Samp


## 画像の回転＋明るさ＋歪み

In [14]:
# 画像の回転＋明るさ＋彩度＋歪み
def rotate_and_adjust_skrw(img_path, prob = 0.5, max_left = 25, max_right = 25, min_factor_adjust = 0.3, max_factor_adjust = 1.7, \
                                 mag = 1):
    """
    画像を回転させる水増し処理
    
    img_path:str
        水増し処理を行いたい画像群のパス
    prob:double
        画像を回転させる確率
    max_left:int
        左に傾ける角度の最大値
    max_right:int
        右に傾ける角度の最小値
    min_factor_adjust:double
        明るさの最小値
    max_factor_adjust:double
        明るさの最大値
    mag:double
        歪みの程度（おそらく）
    
    """
    
    # パイプラインを作成する。
    p = Augmentor.Pipeline(img_path)
    
    # ランダムな回転を実施
    p.rotate(probability = prob, max_left_rotation = max_left, max_right_rotation = max_right)
    #明るさの変更
    p.random_brightness(probability=prob, min_factor=min_factor_adjust, max_factor=max_factor_adjust)
    #歪みを実施
    p.skew(probability=prob, magnitude=mag)
    
    p.sample(1200)

In [15]:
rotate_and_adjust_skrw(train_path)
#適切な形にパス名を変更
rename_path(augmentation_output_path, rename_output_path)

Processing <PIL.Image.Image image mode=RGB size=159x180 at 0x17DC86C55F8>:   0%| | 1/1200 [00:00<02:43,  7.36 Samples/s

Initialised with 603 image(s) found.
Output directory set to ../../Desktop/Train\output.

Processing <PIL.Image.Image image mode=RGB size=146x200 at 0x17DC8784668>: 100%|█| 1200/1200 [00:04<00:00, 255.78 Sampl


## 画像の回転＋明るさ＋歪み＋切り抜き

In [18]:
# 画像の回転＋明るさ＋彩度＋切り抜き
def rotate_and_adjust_skrw_crop(img_path, prob = 0.5, max_left = 25, max_right = 25, min_factor_adjust = 0.3, max_factor_adjust = 1.7, \
                                 mag = 1, percent_crop_area = 0.7):
    """
    画像を回転させる水増し処理
    
    img_path:str
        水増し処理を行いたい画像群のパス
    prob:double
        画像を回転させる確率
    max_left:int
        左に傾ける角度の最大値
    max_right:int
        右に傾ける角度の最小値
    min_factor_adjust:double
        明るさの最小値
    max_factor_adjust:double
        明るさの最大値
    mag:double
        歪みの程度（おそらく）
    percent_crop_area:double
        元画像のどの程度の面積を切り抜くか
    
    """
    
    # パイプラインを作成する。
    p = Augmentor.Pipeline(img_path)
    
    # ランダムな回転を実施
    p.rotate(probability = prob, max_left_rotation = max_left, max_right_rotation = max_right)
    #明るさの変更
    p.random_brightness(probability=prob, min_factor=min_factor_adjust, max_factor=max_factor_adjust)
    #歪みを実施
    p.skew(probability=prob, magnitude=mag)
    #くりぬき実施
    p.crop_random(probability=prob, percentage_area=percent_crop_area, randomise_percentage_area=True)
    
    p.sample(1500)

In [19]:
rotate_and_adjust_skrw_crop(train_path)
#適切な形にパス名を変更
rename_path(augmentation_output_path, rename_output_path)

Executing Pipeline:   0%|                                                               | 0/1500 [00:00<?, ? Samples/s]

Initialised with 603 image(s) found.
Output directory set to ../../Desktop/Train\output.

Processing <PIL.Image.Image image mode=RGB size=49x37 at 0x17DC87DA4E0>: 100%|█| 1500/1500 [00:05<00:00, 293.23 Samples


## 画像の回転＋明るさ＋歪み＋切り抜き＋反転

In [6]:
# 画像の回転＋明るさ＋切り抜き＋反転
def rotate_and_adjust_skrw_crop_mirroring(img_path, prob = 0.5, max_left = 180, max_right = 180, min_factor_adjust = 0.3, max_factor_adjust = 1.7, \
                                 mag = 1, percent_crop_area = 0.7):
    """
    画像を回転させる水増し処理
    
    img_path:str
        水増し処理を行いたい画像群のパス
    prob:double
        画像を回転させる確率
    max_left:int
        左に傾ける角度の最大値
    max_right:int
        右に傾ける角度の最小値
    min_factor_adjust:double
        明るさの最小値
    max_factor_adjust:double
        明るさの最大値
    mag:double
        歪みの程度（おそらく）
    percent_crop_area:double
        元画像のどの程度の面積を切り抜くか
    
    """
    
    # パイプラインを作成する。
    p = Augmentor.Pipeline(img_path)
    
    # ランダムな回転を実施
    p.rotate_without_crop(probability = prob, max_left_rotation = max_left, max_right_rotation = max_right)
    #明るさの変更
    p.random_brightness(probability=prob, min_factor=min_factor_adjust, max_factor=max_factor_adjust)
    #歪みを実施
    #p.skew(probability=prob, magnitude=mag)
    #くりぬき実施
    p.crop_random(probability=prob, percentage_area=percent_crop_area, randomise_percentage_area=False)
    #反転
    p.flip_random(probability=prob)
    
    p.sample(50000)

In [7]:
rotate_and_adjust_skrw_crop_mirroring(train_path)
#適切な形にパス名を変更
rename_path(augmentation_output_path, rename_output_path)

Initialised with 999 image(s) found.
Output directory set to Train\output.

Processing <PIL.Image.Image image mode=RGB size=720x480 at 0x25376880D00>:   9%| | 4610/50000 [01:31<17:14, 43.90 Sampl

In [9]:
rename_path(augmentation_output_path, rename_output_path)

## 画像の回転＋明るさ＋歪み＋切り抜き+弾性歪み

In [11]:
# 画像の回転＋明るさ＋切り抜き+弾性歪み
def rotate_and_adjust_skrw_crop_erasing(img_path, prob = 0.5, max_left = 25, max_right = 25, min_factor_adjust = 0.3, max_factor_adjust = 1.7, \
                                 mag = 1, percent_crop_area = 0.7, grid = 5, mag_d = 10):
    """
    画像を回転させる水増し処理
    
    img_path:str
        水増し処理を行いたい画像群のパス
    prob:double
        画像を回転させる確率
    max_left:int
        左に傾ける角度の最大値
    max_right:int
        右に傾ける角度の最小値
    min_factor_adjust:double
        明るさの最小値
    max_factor_adjust:double
        明るさの最大値
    mag:double
        歪みの程度（おそらく）
    percent_crop_area:double
        元画像のどの程度の面積を切り抜くか
    grid:int
        弾性歪み時の長方形の数
    mag_d:int
        弾性歪みの程度
    
    """
    
    # パイプラインを作成する。
    p = Augmentor.Pipeline(img_path)
    
    # ランダムな回転を実施
    p.rotate_without_crop(probability = prob, max_left_rotation = max_left, max_right_rotation = max_right)
    #明るさの変更
    p.random_brightness(probability=prob, min_factor=min_factor_adjust, max_factor=max_factor_adjust)
    #歪みを実施
    p.skew(probability=prob, magnitude=mag)
    #くりぬき実施
    p.crop_random(probability=prob, percentage_area=0.7, randomise_percentage_area=True)
    #反転
    p.flip_random(probability=prob)
    #弾性歪み
    p.random_distortion(probability=prob, grid_width = grid, grid_height = grid, magnitude=mag_d)
    
    p.sample(10000)

In [12]:
rotate_and_adjust_skrw_crop_erasing(train_path)
#適切な形にパス名を変更
rename_path(augmentation_output_path, rename_output_path)

Initialised with 999 image(s) found.
Output directory set to Train\output.

Processing <PIL.Image.Image image mode=RGB size=306x196 at 0x14A0772B160>:  65%|▋| 6547/10000 [01:15<00:06, 518.95 Samp

In [13]:
rename_path(augmentation_output_path, rename_output_path)

## 画像の回転＋明るさ＋歪み+弾性歪み

In [6]:
# 画像の回転＋明るさ+弾性歪み
def rotate_and_adjust_skrw_erasing(img_path, prob = 0.5, max_left = 180, max_right = 180, min_factor_adjust = 0.3, max_factor_adjust = 1.7, \
                                 mag = 1, grid = 5, mag_d = 3):
    """
    画像を回転させる水増し処理
    
    img_path:str
        水増し処理を行いたい画像群のパス
    prob:double
        画像を回転させる確率
    max_left:int
        左に傾ける角度の最大値
    max_right:int
        右に傾ける角度の最小値
    min_factor_adjust:double
        明るさの最小値
    max_factor_adjust:double
        明るさの最大値
    mag:double
        歪みの程度（おそらく）
    grid:int
        弾性歪み時の長方形の数
    mag_d:int
        弾性歪みの程度
    """
    
    # パイプラインを作成する。
    p = Augmentor.Pipeline(img_path)
    
    # ランダムな回転を実施
    p.rotate_without_crop(probability = prob, max_left_rotation = max_left, max_right_rotation = max_right)
    #明るさの変更
    p.random_brightness(probability=prob, min_factor=min_factor_adjust, max_factor=max_factor_adjust)
    #歪みを実施
    p.skew(probability=prob, magnitude=mag)
    #くりぬき実施
    p.crop_random(probability=prob, percentage_area=0.7, randomise_percentage_area=True)
    #反転
    p.flip_random(probability=prob)
    #弾性歪み
    p.random_distortion(probability=prob, grid_width = grid, grid_height = grid, magnitude=mag_d)
    
    p.sample(100000)

In [7]:
rotate_and_adjust_skrw_erasing(train_path)
#適切な形にパス名を変更
rename_path(augmentation_output_path, rename_output_path)

Initialised with 999 image(s) found.
Output directory set to Train\output.

Processing <PIL.Image.Image image mode=RGB size=42x39 at 0x1EDBD4E2C10>:  23%|▏| 22991/100000 [03:12<07:30, 171.00 SampIOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

